In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
import numpy as np

from btb.tuning.hyperparams import IntHyperParam, FloatHyperParam
from btb.tuning.tunable import Tunable
from btb.tuning.tuners import UniformTuner
from btb.tuning.tuners.gaussian_process import GaussianProcessTuner

In [5]:
hyperparams = {
    'x': FloatHyperParam(-5, 5),
    'y': FloatHyperParam(-5, 5)
}
tunable = Tunable(hyperparams)

In [9]:
class FakeTuner:   
    def sample(self):
        return -5 + np.random.random() * 10
    
    def propose(self):
        return {'x': self.sample(), 'y': self.sample()}
    
    def record(self, proposal, score):
        pass

In [11]:
fake = FakeTuner()

In [12]:
%%timeit

fake.propose()

802 ns ± 43.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%%timeit

proposal = fake.propose()
score = np.random.random()
fake.record(proposal, score)

1.09 µs ± 7.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
uf = UniformTuner(tunable)

In [15]:
%%timeit -n 100

proposal = fake.propose()
score = np.random.random()
uf.record(proposal, score)

571 µs ± 345 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit -n 100

uf.propose(allow_duplicates=True)

695 µs ± 342 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit -n 100

uf.propose(allow_duplicates=False)

697 µs ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
gp = GaussianProcessTuner(tunable)

In [19]:
%%timeit -n 100

proposal = fake.propose()
score = np.random.random()
gp.record(proposal, score)

The slowest run took 14.76 times longer than the fastest. This could mean that an intermediate result is being cached.
22.4 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit -n 100

gp.propose(allow_duplicates=True)

48.7 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit -n 100

gp.propose(allow_duplicates=False)

53.5 ms ± 3.67 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
